# SK-70D13

Updating the dataset: D098100100000

* I don't see a spectrum in the LWRS aperture, but the background is oversubtracted for the SiC 1B spectrum, so I modified the background regions and re-extracted all of the spectra. 
* SiC 1B is better, if not perfect.
* There is no signal in the SiC 2 channel, so I omitted it from the final spectrum.
* Finally, the star's STIS spectrum is slightly brighter than its FUSE spectrum, so I scaled the FUSE spectrum to match.

COMMENT File updated 30 January 2023 <br>
COMMENT Spectrum scaled by 1.23 to match STIS.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.01

# Normalize all spectra, so fluxes are of order unity.
norm = 1E12

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset (in pixels) between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Function to read a regular FUSE spectral file.

def read_fuse(filename):
    f = fits.open(filename)
    data = f[1].data
    f.close()
    spec = make_spectrum (data)
    return data, spec

In [ ]:
# Read header keywords from FUSE *all* file.

filename = 'D098100100000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# Target is in the MDRS aperture.  The guide channel is LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Before proceeding, have a look at SiC 1B.

f, ax = plt.subplots()  
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.legend()

# The background is oversubtracted.  Let's try to fix this.

In [ ]:
# It's not apparent in the detector images, but let's assume that a faint
# star in the LWRS aperture was included in the background calculation. 
# I have re-defined the background regions and re-extracted all of the spectra.

lif1a_data, lif1a = read_fuse('D09810010001alif2ttagfcal.fit')
lif1b_data, lif1b = read_fuse('D09810010001blif2ttagfcal.fit')
lif2a_data, lif2a = read_fuse('D09810010002alif2ttagfcal.fit')
lif2b_data, lif2b = read_fuse('D09810010002blif2ttagfcal.fit')
sic1a_data, sic1a = read_fuse('D09810010001asic2ttagfcal.fit')
sic1b_data, sic1b = read_fuse('D09810010001bsic2ttagfcal.fit')
sic2a_data, sic2a = read_fuse('D09810010002asic2ttagfcal.fit')
sic2b_data, sic2b = read_fuse('D09810010002bsic2ttagfcal.fit')

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Did we solve the problem with the background subtraction?

f, ax = plt.subplots()  
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.legend()

# Well, it's better.  Let's continue.

In [ ]:
# Plot the smoothed spectra.  

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_ylim([0,2.5])

# The LiF 2B and SiC 1A spectra should be rescaled.  
# SiC 2B is hopeless and we will not consider it further.

In [ ]:
# Rescale the LiF 1B and SiC 1A channels to match LiF 1A.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)

# Compute ratio of their fluxes.

mean = sub_lif1a.mean()
scale_lif2 = mean/sub_lif2b.mean()
print ('Scale LiF 2B by', scale_lif2)
scale_sic1 = mean/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1)

In [ ]:
# Scale the spectra.  We will assume that these scale factors apply to the long- and short-wavelength segments, too.

lif2a *= scale_lif2
lif2b *= scale_lif2
sic1a *= scale_sic1
sic1b *= scale_sic1

lif2a_bsmooth *= scale_lif2
lif2b_bsmooth *= scale_lif2
sic1a_bsmooth *= scale_sic1
sic1b_bsmooth *= scale_sic1

In [ ]:
# This looks reasonable, though SiC 1A is pretty noisy.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A')
ax.legend()
ax.set_ylim([-0.25,2.5])

In [ ]:
# Compare LiF 1B with a STIS spectrum of the same star.

filename = 'oeuc4s020_x1d.fits'
stis = Spectrum1D.read(filename, format="HST/STIS")
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.set_xlim([1140,1200])
ax.set_ylim([0,2.5])
ax.legend()

print ('Target:  ', fits.getval(filename, 'TARGNAME'))
print ('Aperture:', fits.getval(filename, 'APERTURE'))

# STIS used a small aperture, so spectrum is probably not contaminated.

# Note that the rescaled LiF 2A spectrum matches LiF 1B pretty well.

In [ ]:
# We'll compute a scale factor between FUSE and STIS and apply it to the whole spectrum at the end.

# Select a spectral region.

region = SpectralRegion(1155*u.AA, 1165*u.AA)
sub_stis = extract_region(stis, region)
sub_lif1b = extract_region(lif1b, region)

# Compute ratio of their fluxes.

scale_entire_spectrum = norm * sub_stis.mean()/sub_lif1b.mean()
print ('Scale FUSE spectrum by', scale_entire_spectrum)

In [ ]:
# This looks fine.

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.step(lif1b_bsmooth.spectral_axis, scale_entire_spectrum * lif1b_bsmooth.flux, label='LiF 2A') 
ax.set_xlim([1140,1200])
ax.set_ylim([0,2.5])
ax.legend()

In [ ]:
# Which spectrum has the higher S/N at long wavelengths?

f, ax = plt.subplots()  
ax.step(lif1b.spectral_axis, lif1b_data['flux']/lif1b_data['error'], label='LiF 1B') 
ax.step(lif2a.spectral_axis, lif2a_data['flux']/lif2a_data['error'], label='LiF 2A') 
ax.set_xlim([1090,1105])
ax.legend()

# LiF 1B has the higher S/N.  Use it instead of LiF 2A where available.

In [ ]:
# We rescaled SiC 1B without checking it.  Let's check it now.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.legend()
ax.set_xlim([970,1000])
ax.set_ylim([-0.5,2.5])

# The scale factor is about right.  Given the low S/N, I won't try anything fancier. 

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A.

shift_lif2 = compute_shift(lif2b_data, lif1a_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)

print ('Shift LiF 2 by ', shift_lif2, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')

In [ ]:
# Shift all six spectra.

shift_lif2 *= WPC
shift_sic1 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
lif2a_shift = Spectrum1D(spectral_axis=lif2a.spectral_axis + shift_lif2 * u.AA, flux=lif2a.flux)
lif2b_shift = Spectrum1D(spectral_axis=lif2b.spectral_axis + shift_lif2 * u.AA, flux=lif2b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,2.5])

In [ ]:
# Lacking the SiC 2 channel, we will have to get creative to cover the full bandpass.

filename = 'D098100100000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 987.99))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 987.99))
data['flux'][g] = float(scale_sic1) * sic1b_data['flux'][h]
data['error'][g] = float(scale_sic1) * sic1b_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 988) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 988) & (lif1a_wave < 1082.5))
data['flux'][g] = lif1a_data['flux'][h]
data['error'][g] = lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1087.49))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1087.49))
data['flux'][g] = float(scale_sic1) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1) * sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087.5) & (data['wave'] < 1094))
h = np.where((lif2a_wave > 1087.5) & (lif2a_wave < 1094))
data['flux'][g] = float(scale_lif2) * lif2a_data['flux'][h]
data['error'][g] = float(scale_lif2) * lif2a_data['error'][h]

# Scale LiF 1B
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1094) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1094) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = lif1b_data['flux'][h]
data['error'][g] = lif1b_data['error'][h]

# Scale spectrum to match STIS
data['flux'] *= float(scale_entire_spectrum)
data['error'] *= float(scale_entire_spectrum)

hdr['comment'] = ''
hdr['comment'] = 'File updated 30 January 2023'
hdr['comment'] = 'Spectrum scaled by {0:.2f} to match STIS.'.format(scale_entire_spectrum)

f.writeto('level0_D098100100000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'D098100100000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 

filename = 'level0_D098100100000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,5E-12])

In [ ]:
# Let's look more closely at the region sampled only by the SiC channels.

f, ax = plt.subplots()  
ax.step(new['wave'], new['flux'], label='NEW FLUX')
ax.step(old['wave'], old['flux'], label='OLD FLUX') 
ax.legend()
ax.set_xlim([1075,1100])
ax.set_ylim([0,5E-12])

# Noisy, but reasonable.